In [36]:
!pip uninstall tensorflow -y


Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0


In [37]:
!pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.8/286.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found e

In [1]:
import tensorflow as tf
print(tf.__version__)


2.16.1


In [3]:
!pip install keras

In [7]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
# Load the dataset
df = pd.read_csv("Dataset.csv", header=None)

In [10]:
# Renaming columns
df.columns = ['email_type', 'content']


In [11]:
# Check for invalid label values
invalid_labels = df[~df['email_type'].isin(range(1, 19))]
if not invalid_labels.empty:
    print("Invalid label values found:")
    print(invalid_labels)

Invalid label values found:
   email_type content
0           0       1


In [12]:
# Remove rows with invalid label values
df = df[df['email_type'].isin(range(1, 19))]

In [13]:
# Data preprocessing
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove special characters (retain numbers)
    text = ' '.join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
    return text

df['content'] = df['content'].apply(preprocess_text)

In [14]:
# Tokenization and Padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['content'])
X_seq = tokenizer.texts_to_sequences(df['content'])
max_length = 500
X_pad = pad_sequences(X_seq, maxlen=max_length, padding='post')


In [15]:
# Update labels
y = df['email_type'].values - 1  # Convert labels to 0-based indices

In [16]:
# Split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_pad, y, test_size=0.2, random_state=42)

In [18]:
# Define the model
embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1

model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim))
model.add(Bidirectional(LSTM(128)))
model.add(Dropout(0.5))
model.add(Dense(18, activation='softmax'))


In [19]:
# Compile the model
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=3)
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True)
history = model.fit(X_train, y_train, epochs=45, batch_size=32, validation_data=(X_test, y_test),
                    callbacks=[early_stop, checkpoint])

Epoch 1/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 126s 3s/step - accuracy: 0.1882 - loss: 2.7358 - val_accuracy: 0.6056 - val_loss: 1.5809
Epoch 2/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 140s 3s/step - accuracy: 0.6735 - loss: 1.2027 - val_accuracy: 0.9444 - val_loss: 0.3690
Epoch 3/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.9415 - loss: 0.3288 - val_accuracy: 0.9639 - val_loss: 0.1637
Epoch 4/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 139s 2s/step - accuracy: 0.9839 - loss: 0.1114 - val_accuracy: 0.9639 - val_loss: 0.1156
Epoch 5/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 144s 2s/step - accuracy: 0.9869 - loss: 0.0744 - val_accuracy: 0.9722 - val_loss: 0.0909
Epoch 6/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 147s 3s/step - accuracy: 0.9926 - loss: 0.0427 - val_accuracy: 0.9694 - val_loss: 0.1022
Epoch 7/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 135s 2s/step - accuracy: 0.9995 - loss: 0.0197 - val_accuracy: 0.9694 - val_loss: 0.0757
Epoch 8/45
45/45 ━━━━━━━━━━━━━━━━━━━━ 143s 2s/step - accuracy: 0.9977 - loss: 0.0197 - val_accuracy: 0.9722 - v

In [22]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}, Test Accuracy: {accuracy}")


12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 604ms/step - accuracy: 0.9800 - loss: 0.0644
Test Loss: 0.09080126881599426, Test Accuracy: 0.9722222089767456


In [23]:
# Save tokenizer and model
model.save("email_classification_model.h5")
tokenizer_json = tokenizer.to_json()
with open("tokenizer.json", "w") as json_file:
    json_file.write(tokenizer_json)


ValueError: Unable to synchronously create dataset (name already exists)